# Planar Peg-In-Hole

In this notebook, we will become familiar with the famous peg-in-hole task for robotic manipulation. This task is hard because applying wrong or too strong forces quickly result in the peg getting jammed, and therefore position-based controllers tend to do poorly. Therefore, we will be using indirect force control: stiffness control in the task space!

**Learning Objectives:**
1. Implement stiffness control for a floating peg.
2. Implement operational space stiffness control for an iiwa manipulating a peg

**What you'll build:** A complete peg-in-hole insertion system featuring both a simplified planar robot and a full IIWA14 arm performing precision insertion tasks with force control!

---

In [ ]:
import time
from typing import Literal

import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    BasicVector,
    ContactVisualizer,
    ContactVisualizerParams,
    Context,
    DiagramBuilder,
    FixedOffsetFrame,
    InputPort,
    InverseKinematics,
    JacobianWrtVariable,
    LeafSystem,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    ModelInstanceIndex,
    MultibodyPlant,
    MultibodyPositionToGeometryPose,
    OutputPort,
    Parser,
    PiecewisePolynomial,
    PrismaticJoint,
    RevoluteJoint,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    SceneGraph,
    Simulator,
    Solve,
    SpatialInertia,
    StartMeshcat,
    TrajectorySource,
    UnitInertia,
)

from manipulation import running_as_notebook
from manipulation.scenarios import AddMultibodyTriad, SetColor
from manipulation.station import AppendDirectives, LoadScenario, MakeHardwareStation
from manipulation.utils import ConfigureParser, FindResource, RenderDiagram

In [ ]:
meshcat = StartMeshcat()

In this notebook, we will be using the planar iiwa. Let us therefore set the meshcat render mode to 2D:

In [ ]:
meshcat.Set2dRenderMode(xmin=-0.25, xmax=1.5, ymin=-0.1, ymax=1.3)

We have provided some helper functionality for you below, that we will use to set up the scene. It will be useful to have looked through these functions later (at least read the docstrings)!

In [ ]:
PEG_LENGTH = 0.2  # m


def add_planar_peg_to_plant(
    plant: MultibodyPlant, peg_frame_offset: float = 0.0
) -> ModelInstanceIndex:
    """
    Adds a planar peg to the plant, with three actuators: x-axis, z-axis, and pitch
    (rotation around y-axis). This lets us directly command forces and torques along
    these axis.

    Notice that the joint space and task space for this peg are the same!

    Returns a reference to the ModelInstanceIndex of the peg.
    """

    parser = Parser(plant)
    ConfigureParser(parser)

    peg = parser.AddModelsFromUrl("package://manipulation/peg_in_hole/peg.urdf")[0]

    # We define several "false bodies" to make it possible to add actuators directly on
    # each planar DOF. You don't need to understand this code!

    peg_false_body1 = plant.AddRigidBody(
        "false_body1",
        peg,
        SpatialInertia(0, [0, 0, 0], UnitInertia(0, 0, 0)),
    )
    peg_false_body2 = plant.AddRigidBody(
        "false_body2",
        peg,
        SpatialInertia(0, [0, 0, 0], UnitInertia(0, 0, 0)),
    )
    peg_x = plant.AddJoint(
        PrismaticJoint(
            "peg_x",
            plant.world_frame(),
            plant.GetFrameByName("false_body1"),
            [1, 0, 0],
            -10,
            10,
        )
    )
    plant.AddJointActuator("peg_x", peg_x)
    peg_z = plant.AddJoint(
        PrismaticJoint(
            "peg_z",
            plant.GetFrameByName("false_body1"),
            plant.GetFrameByName("false_body2"),
            [0, 0, 1],
            -10,
            10,
        )
    )
    peg_z.set_default_translation(0.0)
    plant.AddJointActuator("peg_z", peg_z)
    peg_frame = plant.AddFrame(
        FixedOffsetFrame(
            "peg_tip_frame",
            plant.GetFrameByName("peg_body_link", peg),
            RigidTransform([0, 0, peg_frame_offset]),
        )
    )
    peg_theta = plant.AddJoint(
        RevoluteJoint(
            "peg_theta",
            plant.GetFrameByName("false_body2"),
            peg_frame,
            [0, 1, 0],
            -10,
            10,
        )
    )
    plant.AddJointActuator("peg_theta", peg_theta)

    return peg


def add_table_to_plant(plant: MultibodyPlant) -> ModelInstanceIndex:
    parser = Parser(plant)
    ConfigureParser(parser)

    table = parser.AddModelsFromUrl(
        "package://manipulation/hydro/extra_heavy_duty_table_surface_only_collision.sdf"
    )[0]
    plant.WeldFrames(
        plant.world_frame(),
        plant.GetFrameByName("link", table),
        RigidTransform([0.5, 0, -0.7645]),
    )
    return table


def add_planar_hole_to_plant(
    plant: MultibodyPlant, table: ModelInstanceIndex
) -> ModelInstanceIndex:
    parser = Parser(plant)
    ConfigureParser(parser)
    hole = parser.AddModelsFromUrl(
        "package://manipulation/peg_in_hole/hole_chamfered.sdf"
    )[0]
    plant.WeldFrames(
        plant.GetFrameByName("link", table),
        plant.GetFrameByName("hole_chamfered_body_link", hole),
        RigidTransform([0, 0.125, 0.7645]),
    )
    return hole


def add_setpoint_visualization(
    builder: DiagramBuilder,
    meshcat: MeshcatVisualizer,
    pos_traj_port: OutputPort,
    peg_frame_offset: float = 0.0,
) -> None:
    """
    This function adds a transparent peg that visualizes the commanded position of the peg.
    """
    # Use the controller plant to visualize the set point geometry.
    controller_scene_graph = builder.AddSystem(SceneGraph())
    controller_plant = MultibodyPlant(time_step=0.005)
    controller_plant.RegisterAsSourceForSceneGraph(controller_scene_graph)
    add_planar_peg_to_plant(controller_plant, peg_frame_offset)
    controller_plant.Finalize()
    SetColor(
        controller_scene_graph,
        color=[1.0, 0.0, 0.0, 0.2],
        source_id=controller_plant.get_source_id(),
    )
    AddMultibodyTriad(
        controller_plant.GetFrameByName("peg_tip_frame"), controller_scene_graph
    )
    controller_vis = MeshcatVisualizer.AddToBuilder(
        builder,
        controller_scene_graph,
        meshcat,
        MeshcatVisualizerParams(prefix="controller"),
    )
    controller_vis.set_name("controller meshcat")
    positions_to_poses = builder.AddSystem(
        MultibodyPositionToGeometryPose(controller_plant)
    )
    builder.Connect(
        positions_to_poses.get_output_port(),
        controller_scene_graph.get_source_pose_port(controller_plant.get_source_id()),
    )
    builder.Connect(pos_traj_port, positions_to_poses.get_input_port())


def get_peg_insertion_trajectory(
    nominal_pos_xz: np.ndarray,
    peg_frame_offset: float = 0.0,
    trans_pert: float = 0.03,
    rot_pert: float = 0.09,
) -> tuple[np.ndarray, PiecewisePolynomial]:
    """
    This function returns a linear trajectory and a start position, given a nominal planar
    (x, z) position, with an added random perturbation.

    The returned trajectory is such that the commanded peg penetrates the table, which
    is desirable when using stiffness control and we want the peg to be pushed all the way
    down into the hole.

    @param nominal_pos_xz: nominal (x, z) position for the **COM** of the peg.
    @param peg_frame_offset: accounts for moving the peg frame around, such that we always
           define the trajectory in terms of the COM of the peg.

    """
    x_nom, z_nom = nominal_pos_xz
    pos_initial = np.array(
        [x_nom, z_nom + peg_frame_offset, 0.0]
    )  # (pos_x, pos_z, pitch)
    pert = np.random.uniform(
        [-trans_pert, -trans_pert, -rot_pert], [trans_pert, trans_pert, rot_pert]
    )
    start_pos = pos_initial + pert

    z_pos_end = -PEG_LENGTH / 2 + peg_frame_offset

    traj = PiecewisePolynomial.FirstOrderHold(
        [0, 5.0],
        np.array([start_pos, [x_nom, z_pos_end, 0]]).T,
    )
    return start_pos, traj


def get_peg_frame_offset(placement: Literal["back", "center", "tip"]) -> float:
    """
    Helper function for computing the offset along the peg-aligned axis to place the
    frame either at the back, center, or tip of the peg.

    NOTE: For `tip`, we actually place the frame a bit in front of the tip of the peg,
          which gives better performance for the peg-in-hole task.

    You should expect the `tip` setting to work best for the peg-in-hole task.
    """
    if placement == "center":
        return 0.0
    elif placement == "back":
        return PEG_LENGTH / 2
    else:  # tip
        OFFSET = PEG_LENGTH / 4  # Place the frame a bit ahead of the peg tip
        return -PEG_LENGTH / 2 - OFFSET

---

## Part 1: Floating peg

Great!

In this first part of the notebook, we will control a floating peg in the plane.
Here is some relevant information about the 2D plant:

- $q = [x, z, \theta]$, i.e. the x-position, z-position, and the pitch (rotation about the y-axis)
- $\dot q = [\dot x, \dot z, \dot \theta]$, i.e. the corresponding velocities of the peg
- $ u = [f_x, f_z, \tau_\theta]$ i.e. force in x-direction, force in z-direction, and torque about the y-axis.

The scene looks like this:

![planar-floating-peg](https://raw.githubusercontent.com/RussTedrake/manipulation/master/book/figures/exercises/planar_peg_in_hole_floating.png)


Your first task will be to implement a stiffness controller for this plant. Notice that for the floating peg, the task space and joint space are identical, hence there is no difference between joint space and operational/task space control!

Below we have implemented the skeleton for a `StiffnessController`. It is your job to finish it!

**Reference**: To complete this part, you will have had to read [Chapter 8](https://manipulation.csail.mit.edu/force.html) in the textbook.


In [ ]:
class StiffnessController(LeafSystem):
    def __init__(self, plant: MultibodyPlant, k_p: np.ndarray, k_d: np.ndarray) -> None:
        super().__init__()

        assert k_p.shape == (3,)
        assert k_d.shape == (3,)
        self._plant = plant
        self._K_p = np.diag(k_p)
        self._K_d = np.diag(k_d)

        # TODO: Make a new context from the plant and store it as a private variable.
        #       (We will need a copy of the context for the plant for this controller!)

        # TODO: Declare input ports for desired position and velocity

        # TODO: Declare input port for the state

        # TODO: Declare output port for the generalized actuation forces

    def OutputForces(self, context: Context, output: BasicVector) -> None:
        pass  # TODO: Remove when you implement

        # TODO: Get the state from the input port

        # TODO: Get the desired position and velocities

        # TODO: Compute the current generalized gravitational force
        # HINT 1: First use self._plant.SetPositionsAndVelocities(...) to update the
        #       private plant context copy you saved in the constructor
        # HINT 2: Then use self._plant.CalcGravityGeneralizedForces(...)

        # TODO: Compute the input as:
        # u = K_p (q_des - q) + K_d (q_dot_des - q_dot) - tau_gravity

        # TODO: Set the output from u

Great. Now it is time to test our controller. We have provided you with the start of a function below. It is up to you to finish it!

**Placement of peg frame:**
As noted in the [peg-in-hole section in Chapter 8](https://manipulation.csail.mit.edu/force.html#section5), it matters what point the stiffness controller tries to rotate the peg around. This is what the `peg_frame_placement` below does: it places the reference frame used for stiffness control either at the back, the center or at the tip of the peg. Play around with different placements and see the effect. You will most likely find that it is easier when the frame is placed at the **tip** of the peg!

In [ ]:
def run_simulation(
    k_p: np.ndarray,
    k_d: np.ndarray,
    peg_frame_placement: Literal["back", "center", "tip"] = "center",
) -> None:
    """
    Runs a simulation with your StiffnessController.

    @param peg_frame_placement: Where to place the center of rotation used by the
           stiffness controller.
    """

    # Compute the numeric offset of the peg frame based on desired frame placement
    peg_frame_offset = get_peg_frame_offset(peg_frame_placement)

    # Set up the scene for the planar floating peg.
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.005)
    peg = add_planar_peg_to_plant(plant, peg_frame_offset=peg_frame_offset)
    table = add_table_to_plant(plant)
    hole = add_planar_hole_to_plant(plant, table)
    plant.Finalize()

    # Visualize the peg tip
    AddMultibodyTriad(plant.GetFrameByName("peg_tip_frame"), scene_graph)

    # Set up meshcat
    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    ContactVisualizer.AddToBuilder(
        builder,
        plant,
        meshcat,
        ContactVisualizerParams(radius=0.005, newtons_per_meter=10.0),
    )

    # Set up trajectory references for the peg
    start_pos, traj = get_peg_insertion_trajectory(
        np.array([0.5, 0.35]), peg_frame_offset
    )
    pos_ref = builder.AddSystem(TrajectorySource(traj))
    vel_ref = builder.AddSystem(TrajectorySource(traj.derivative()))
    add_setpoint_visualization(
        builder, meshcat, pos_ref.get_output_port(), peg_frame_offset=peg_frame_offset
    )

    # TODO: Add your controller to builder, connect the position and velocity references,
    #       state input from the plant, and controller output to the actuation of the
    #       plant.

    # Build diagram and simulate
    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    plant_context = plant.GetMyContextFromRoot(context)

    plant.SetPositions(plant_context, start_pos)

    if not running_as_notebook:
        simulator.AdvanceTo(0.01)
    else:
        simulator.set_target_realtime_rate(1.0)
        meshcat.StartRecording()
        simulator.AdvanceTo(7.5)  # feel free to change the end time
        meshcat.StopRecording()
        meshcat.PublishRecording()

Great. Now, you just need to find some good controller gains. Run the cell below to see your controller in action for various initial conditions.

Remember to try different frame placements. Most likely, you'll find that is it better to have the frame placed at the "tip" of the peg, rather than "back" or "center". Experiment and see for yourself!

In [ ]:
# TODO: Set some reasonable values for the gains.
# HINT: Start with k_d = 0, and try something small for k_p.
#       Stop increasing k_p when you start to see oscillations, and
#       start setting k_d to something small (for instance around ~0.01).
# HINT: Your values for position and orientation may have to be different!
# HINT: If your peg is going all over the place, then you probably have too high gains
#       (for instance, k_p = 50 )
k_p = np.array([0, 0, 0])
k_d = np.array([0, 0, 0])

np.random.seed(1)  # feel free to change the seed
for _ in range(5 if running_as_notebook else 1):
    run_simulation(
        k_p, k_d, peg_frame_placement="center"
    )  # try changing the peg frame placement

---

## Part 2: Peg-in-hole the full (planar) iiwa arm

Awesome!  We will now do the same, except that we will do it for the whole iiwa arm. Since the joint space and the task space of the iiwa are different (as opposed to the floating peg), we will now have to do *operational space* stiffness control, also known as *task space stiffness control*, or *spatial stiffness control*.

The scene for this part will look like this:

![planar-floating-peg](https://raw.githubusercontent.com/RussTedrake/manipulation/master/book/figures/exercises/planar_peg_in_hole_iiwa.png)

**Reference:** For this, you will have to look at [Chapter 8.3: The General Case](https://manipulation.csail.mit.edu/force.html#section3), and in particular at chapter 8.3.3.

Let us start by clearing meshcat:

In [ ]:
# clear meshcat
meshcat.Delete()

We have provided you with the scenario setup. Simply run the function below to set up the scene.


In [ ]:
def make_scenario_string(
    peg_frame_placement: Literal["back", "center", "tip"] = "center",
) -> str:
    peg_frame_offset = get_peg_frame_offset(peg_frame_placement)
    scenario_string = f"""
directives:
- add_model:
    name: iiwa
    file: package://manipulation/planar_iiwa14_no_collision.urdf
    default_joint_positions:
        iiwa_joint_2: [0.1]
        iiwa_joint_4: [-1.2]
        iiwa_joint_6: [1.6]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: robot_table
    file: package://manipulation/hydro/extra_heavy_duty_table_surface_only_collision.sdf
- add_weld:
    parent: world
    child: robot_table::link
    X_PC:
        translation: [0, 0, -0.7645]
- add_model:
    name: work_table
    file: package://manipulation/hydro/extra_heavy_duty_table_surface_only_collision.sdf
- add_weld:
    parent: world
    child: work_table::link
    X_PC:
        translation: [0.75, 0, -0.7645]
- add_model:
    name: peg
    file: package://manipulation/peg_in_hole/peg.urdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: peg::peg_body_link
    X_PC:
        translation: [0, 0, 0.10]
        rotation: !Rpy {{ deg: [0, 180, -90] }}
- add_frame:
    name: peg_tip_frame
    X_PF:
      base_frame: peg::peg_body_link
      translation: [0, 0, {peg_frame_offset}]
      rotation: !Rpy {{ deg: [0, 0, 0] }}
- add_model:
    name: hole 
    file: package://manipulation/peg_in_hole/hole_chamfered.sdf
- add_weld:
    parent: work_table::link
    child: hole::hole_chamfered_body_link
    X_PC:
        translation: [0, 0.1, 0.7645]
        rotation: !Rpy {{ deg: [0, 0, 0] }}
model_drivers:
    iiwa: !IiwaDriver
      control_mode: torque_only
      desired_kp_gains: [500, 500, 200]
      hand_model_name: wsg
  """
    return scenario_string

In [ ]:
class OperationalStiffnessController(LeafSystem):
    def __init__(self, plant: MultibodyPlant, k_p: np.ndarray, k_d: np.ndarray) -> None:
        super().__init__()

        self._plant = plant
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        assert k_p.shape == (3,)
        assert k_d.shape == (3,)

        self._K_p = np.diag(k_p)
        self._K_d = np.diag(k_d)

        self._torque_max = 8  # [Nm], do not change

        self._P = plant.GetFrameByName("peg_tip_frame")
        self._W = plant.world_frame()

        # These are the free joints for the planar iiwa
        self._free_joint_indices = [
            plant.GetJointByName(j).position_start()
            for j in ("iiwa_joint_2", "iiwa_joint_4", "iiwa_joint_6")
        ]

        # TODO: Create and store a local plant context that we can use for
        #       computations later

        # TODO: Declare input ports for desired task space positions and velocities

        # TODO: Declare input port for the state of the planar iiwa (see the free
        #       joints above).

        # TODO: Declare the output port for the commanded torque

    def CalcTorqueOutput(self, context: Context, output: BasicVector) -> None:
        # TODO: Get the state

        # TODO: Set the position and velocities of the copy of the plant context

        # TODO: Get the desired position and velocity

        # TODO: Compute the spatial jacobian for the peg frame (self._P)
        #       (with respect to q_dot, time derivative of generalized positions)

        # TODO: Pick out the relevant rows and columns for the planar iiwa in
        #       the Jacobian.
        # HINT: Use J[np.ix_(config_idxs, free_joint_idxs)]:
        #  - Rows correspond to configurations (x, z, pitch),
        #    i.e. idxs (3, 5, 1) in the full configuration of the iiwa.
        #  - Columns correspond to self._free_joint_indices
        # We end up with J of shape (3,3).
        # NOTE: The order here matters a lot! Make sure to use the order described above

        # TODO: Extract the planar position of the peg: (x_pos, z_pos, pitch)
        # HINT: Get X_WP with self._plant.CalcRelativeTransform, and use RollPitchYaw
        #       to get the pitch.

        # TODO: Calculate the planar velocity of the peg from q_dot
        # HINT: Use the Jacobian

        # TODO: Compute the desired task space spatial force as:
        # F_u = K_p (p_des - p) + K_d (v_des - v)
        # NOTE: In contrast to in the textbook, we don't need to compensate for the
        #       gravitational force, as Drake's internal iiwa controller already does
        #       this for us.

        # TODO: Convert from task space spatial force to joint space torques
        # HINT: Use the jacobian

        u = np.array([0, 0, 0])
        u_clamped = np.clip(
            u, -self._torque_max, self._torque_max
        )  # do not change this
        output.SetFromVector(u_clamped)

Amazing. Now, for the grand finale! Complete the simulation function below.

In [ ]:
# You do not need to modify this.


def solve_ik(
    plant: MultibodyPlant,
    pose_des: np.ndarray,
    trans_tol: float = 0.0,
    ang_tol: float = 0.0,
) -> np.ndarray:
    """
    Computes the joint angles for the planar iiwa given a desired pose for the peg.

    You will find this useful for computing the starting configuration for the iiwa.
    """
    context = plant.CreateDefaultContext()
    ik = InverseKinematics(plant, context)
    q = ik.q()
    prog = ik.prog()

    P = plant.GetFrameByName("peg_tip_frame")
    W = plant.world_frame()

    x_des, z_des, pitch_des = pose_des
    peg_pos_3d = np.array([x_des, 0, z_des])
    pos_bound_3d = np.array([trans_tol, 10, trans_tol])  # allow any y-position

    ik.AddPositionConstraint(
        frameB=P,
        p_BQ=np.zeros(3),
        frameA=W,
        p_AQ_lower=peg_pos_3d - pos_bound_3d,
        p_AQ_upper=peg_pos_3d + pos_bound_3d,
    )

    R_WP_des = RollPitchYaw([0, pitch_des, 0]).ToRotationMatrix()
    ik.AddOrientationConstraint(
        frameAbar=W,
        R_AbarA=R_WP_des,
        frameBbar=P,
        R_BbarB=RotationMatrix(),
        theta_bound=ang_tol,
    )

    q_seed = np.array([0.1, -1.2, 1.6])
    Q = np.eye(plant.num_positions())
    prog.AddQuadraticErrorCost(Q, q_seed, q)
    prog.SetInitialGuess(q, q_seed)

    result = Solve(prog)
    assert result.is_success()
    return result.GetSolution(q)

In [ ]:
def run_simulation(
    k_p: np.ndarray,
    k_d: np.ndarray,
    peg_frame_placement: Literal["back", "center", "tip"] = "center",
) -> None:
    """
    Runs a simulation with your OperationalStiffnessController

    @param peg_frame_placement: Where to place the center of rotation used by the
          stiffness controller.
    """
    # Set up the scene
    peg_frame_offset = get_peg_frame_offset(peg_frame_placement)
    scenario_string = make_scenario_string(peg_frame_placement)
    scenario = LoadScenario(data=scenario_string)
    builder = DiagramBuilder()
    station = builder.AddSystem(MakeHardwareStation(scenario, meshcat))
    plant = station.GetSubsystemByName("plant")
    scene_graph = station.GetSubsystemByName("scene_graph")
    AddMultibodyTriad(plant.GetFrameByName("peg_tip_frame"), scene_graph)

    # Compute the peg insertion trajectory
    pos_xz_start = np.array([0.75, 0.4])
    start_pose, traj = get_peg_insertion_trajectory(
        pos_xz_start, peg_frame_offset, trans_pert=0.03, rot_pert=0.07
    )
    pos_ref = builder.AddSystem(TrajectorySource(traj))
    vel_ref = builder.AddSystem(TrajectorySource(traj.derivative()))
    add_setpoint_visualization(
        builder, meshcat, pos_ref.get_output_port(), peg_frame_offset
    )

    # TODO: Add your controller with your desired gains and wire it up correctly.
    # NOTE: Your gains may have to be higher for this controller, as we now have the
    #       mass of the entire iiwa robot, not just the peg!

    # TODO: Compute an initial configuration by using the provided IK function above
    q_initial = np.array([0, 0, 0])

    # TODO: Build diagram, extract context and plant_context

    # TODO: Set initial positions according to q_initial

    # TODO: Simulate

### How well does position control work?

First, let us start with seeing how well a position controller would work in this setting. We can achieve this by making our controller very *stiff*, i.e. by picking high gains (technically, with our implementation this gives us stiff position control in the **task space**). Below, we have provided you with some gains for a very stiff controller. Generally, **position control will not work well for the peg-in-hole task**. See for yourself, how well does your position controller work?

#### VERIFICATION IN GRADESCOPE (1/2)

Run your controller for 5 runs by running the cell block below, which will test your controller for slightly different start configurations.
Take one screen recording of the five simulations (meshcat should just reset and run automatically) and upload it to gradescope as an mp4. The file should be (much) smaller than 500MB.

**Video requirement**: We **do not** expect this stiff controller to work. We want you to see for yourself that position control is **not** well suited for a task like this. For this video, it is expected that you upload a video of 5 attempts where the robot jams the peg into the walls of the hole and fails to insert the peg into the hole.

In [ ]:
# Controller gains that result in a *stiff* controller, which resembles position control.
# You should **not** expect this to work!
k_p_stiff = np.array([2000, 2000, 2000])
k_d_stiff = np.array([100, 100, 100])

if running_as_notebook:
    time.sleep(
        5
    )  # change this to have time to start screen recording before the simulation runs!

np.random.seed(2)
for _ in range(5 if running_as_notebook else 1):
    run_simulation(k_p_stiff, k_d_stiff, peg_frame_placement="tip")

### Indirect force control (stiffness control)

Now, let us actually use the task space stiffness controller as we intended. 

#### VERIFICATION IN GRADESCOPE (2/2)

Run your controller for 5 runs by running the cell block below, which will test your controller for slightly different start configurations.
Take one screen recording of the five simulations (meshcat should just reset and run automatically) and upload it to gradescope as an mp4. The file should be (much) smaller than 500MB.

**Video requirement**: Your controller should be able to succesfully insert the peg for the 5/5 trials. It is okay if the peg does not go all the down into the hole, but it should be very close.

In [ ]:
# TODO: Find gains for the stiffness controller. These should be lower than the stiff
#       values we set above!
# NOTE: Your gains may have to be higher for this controller than for the floating peg,
#       as we now have the mass of the entire iiwa robot, not just the peg!
k_p = np.array([0, 0, 0])
k_d = np.array([0, 0, 0])

if running_as_notebook:
    time.sleep(
        5
    )  # change this to have time to start screen recording before the simulation runs!

np.random.seed(0)  # feel free to change the seed!
for _ in range(5 if running_as_notebook else 1):
    run_simulation(k_p, k_d, peg_frame_placement="tip")